In [9]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
sb.set() 

In [3]:
movie_data = pd.read_csv('TMBD Movie Dataset.csv')

We will be using profit and popularity as the main factors of success

In [4]:
profit = pd.DataFrame(movie_data['profit'])
popularity = pd.DataFrame(movie_data['popularity'])

We have filtered out the factors that we identified to have relevance to the success of a movie 

In [22]:
budget = pd.DataFrame(movie_data['budget'])
cast = pd.DataFrame(movie_data['cast'])
director = pd.DataFrame(movie_data['director'])
genres = pd.DataFrame(movie_data['genres'])
runtime = pd.DataFrame(movie_data['runtime'])
production_companies = pd.DataFrame(movie_data['production_companies'])
release_year = pd.DataFrame(movie_data['release_year'])
release_date = pd.DataFrame(movie_data['release_date'])
compiled = movie_data[['popularity', 'profit', 'budget', 'cast', 'director', 'genres', 'production_companies', 'release_year', 'release_date', 'runtime']]

In [23]:
compiled.dropna(axis=0)

,popularity,profit,budget,cast,director,genres,production_companies,release_year,release_date,runtime
0,32.985763,1.363529e+09,150000000.0,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015,2015-06-09,124
1,28.419936,2.284364e+08,150000000.0,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015,2015-05-13,120
2,13.112507,1.852382e+08,110000000.0,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015,2015-03-18,119
3,11.173104,1.868178e+09,200000000.0,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015,2015-12-15,136
4,9.335014,1.316249e+09,190000000.0,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015,2015-04-01,137
...,...,...,...,...,...,...,...,...,...,...
1282,1.549139,1.547778e+08,7000000.0,Roger Moore|Yaphet Kotto|Jane Seymour|Clifton ...,Guy Hamilton,Adventure|Action|Thriller,Eon Productions|Metro-Goldwyn-Mayer (MGM),1973,1973-07-05,121
1283,1.910465,1.301957e+08,11000000.0,Sean Connery|Claudine Auger|Adolfo Celi|Lucian...,Terence Young,Adventure|Action|Thriller,Eon Productions|Metro-Goldwyn-Mayer (MGM),1965,2065-12-16,130
1284,1.778746,7.497449e+07,7000000.0,George Lazenby|Diana Rigg|Telly Savalas|Gabrie...,Peter R. Hunt,Adventure|Action|Thriller,Eon Productions|Metro-Goldwyn-Mayer (MGM)|Danjaq,1969,2069-12-12,142
1285,1.198849,6.970000e+07,300000.0,Donald Pleasence|Jamie Lee Curtis|P.J. Soles|N...,John Carpenter,Horror|Thriller,Compass International Pictures|Falcon Internat...,1978,1978-10-25,91
